In [7]:
data_dir = "/Volumes/Data/Research/Healthy Brain Network/ActiGraph/gt3x_data"
subdirs = ["in process", "processed"]

In [47]:
from datetime import datetime, timedelta
import os
import numpy as np
import pandas as pd
import zipfile

In [33]:
def read_zips(data_dir, subdirs):
    """
    Function to read in all specified raw csvs.
    
    Parameters
    ----------
    data_dir : string
        parent directory
        
    subdirs : list of strings
        subdirectories
        
    Returns
    -------
    dfs : list of 3-tuples of strings, DataFrames, and strings
        ([0], : string
            filename
        [1], : Dataframe
            RAW csv file exported from ActiLife
        [2]) : string
            device serial number
    """
    dfs = list()
    for subdir in subdirs:
        sd = os.path.join(data_dir, subdir)
        for file in os.listdir(sd):
            if file.endswith(".gt3x"):
                zp = os.path.join(sd, file)
                zdict = {}
                try:
                    with zipfile.ZipFile(zp) as z:
                        with z.open("info.txt", "r") as f:
                            for line in f:
                                try:
                                    key, value = line.decode("UTF-8").split(":")
                                    zdict[key] = value[1:-2]
                                except:
                                    continue
                    dfs.append(zdict)
                except:
                    print(zp)
    return(dfs)

In [34]:
z = read_zips(data_dir, subdirs)

/Volumes/Data/Research/Healthy Brain Network/ActiGraph/gt3x_data/in process/5783223 (2017-09-14).gt3x


In [39]:
df = pd.DataFrame.from_dict(z)

In [50]:
df

,Acceleration Max,Acceleration Min,Acceleration Scale,Battery Voltage,Board Revision,Device Type,Dominance,Download Date,Firmware,Last Sample Time,Limb,Sample Rate,Serial Number,Sex,Side,Start Date,Stop Date,Subject Name,Unexpected Resets
0,8.0,-8.0,256.0,3.59,4,wGT3XBT,Non-Dominant,633032067740000000,1.9.2,636391276800000000,Wrist,60,MOS2D17170070,NaN,Left,636373599000000000,636391276800000000,5060956,0
1,8.0,-8.0,256.0,4.18,4,wGT3XBT,Non-Dominant,633038084880000000,1.9.2,636405251400000000,Wrist,60,MOS2D17170084,NaN,Left,636385700400000000,636405251400000000,5082352,0
2,8.0,-8.0,256.0,4.19,4,wGT3XBT,Non-Dominant,633038985810000000,1.9.2,636405251400000000,Wrist,60,MOS2D17170084,NaN,Left,636385700400000000,636405251400000000,5082352,0
3,8.0,-8.0,256.0,3.2,4,wGT3XBT,Non-Dominant,633032064580000000,1.9.2,636416799000000000,Wrist,60,MOS2D17170100,Male,Left,636397122600000000,636416799000000000,5150661,0
4,8.0,-8.0,256.0,3.78,4,wGT3XBT,Non-Dominant,633034833470000000,1.9.2,636390694800000000,Wrist,60,MOS2D17170085,NaN,Left,636372932400000000,636390694800000000,5161062,0
5,8.0,-8.0,256.0,3.46,4,wGT3XBT,Non-Dominant,633032065600000000,1.9.2,636410487600000000,Wrist,60,MOS2D17170097,NaN,Right,636390915600000000,636410487600000000,5226588,0
6,8.0,-8.0,256.0,3.29,4,wGT3XBT,Non-Dominant,633032064760000000,1.9.2,636410946600000000,Wrist,60,MOS2D17170095,Female,Left,636391810200000000,636410946600000000,5257444,0
7,8.0,-8.0,256.0,3.46,4,wGT3XBT,Non-Dominant,633032066270000000,1.9.2,636388072800000000,Wrist,60,MOS2D17170068,NaN,Left,636367703400000000,636388072800000000,5266756,0
8,8.0,-8.0,256.0,3.47,4,wGT3XBT,Non-Dominant,633032066490000000,1.9.2,636412474200000000,Wrist,60,MOS2D17170106,NaN,Right,636391775400000000,636412474200000000,5279473,0
9,8.0,-8.0,256.0,3.42,4,wGT3XBT,Non-Dominant,633032065630000000,1.9.2,636395345400000000,Wrist,60,MOS2D17170087,NaN,Left,636377222400000000,636395345400000000,5315033,0
